In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd '/content/gdrive/My Drive/CSE538/project/text-to-code/code/'

/content/gdrive/My Drive/CSE538/project/text-to-code/code


In [ ]:
ls

beam.py  bleu.py  dataset.py  __pycache__/  run.py  text2code_concode.log


In [ ]:
import torch
torch.__version__

'1.7.0+cu101'

In [3]:
pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 20kB/s 
     |████████████████████████████████| 4.0MB 46.2MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [4]:
!pip install transformers

     |████████████████████████████████| 1.4MB 8.4MB/s 
     |████████████████████████████████| 2.9MB 29.3MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5ed67adbaf610dd7de555ca226602fd0e944ec8aa08938cdbe4d9ffe9e8a8368
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [65]:
ls

dev.json  test1.json  test.json  train_7500.json  train.json


In [55]:
!head -50000 train.json >train_7500.json

In [8]:
! /opt/bin/nvidia-smi

Mon Nov 30 22:44:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [62]:
cd '/content/gdrive/My Drive/CSE538/project/text-to-code/dataset/concode'

/content/gdrive/My Drive/CSE538/project/text-to-code/dataset/concode


# train

In [17]:
!python -m torch.distributed.launch --nproc_per_node=1 run.py \
        --data_dir=../dataset/concode \
        --langs=java \
        --output_dir=../save/concode \
        --pretrain_dir=microsoft/CodeGPT-small-java-adaptedGPT2 \
        --log_file=text2code_concode.log \
        --model_type=gpt2 \
        --block_size=512 \
        --do_train \
        --node_index 0 \
        --gpu_per_node 1 \
        --learning_rate=5e-5 \
        --weight_decay=0.01 \
        --evaluate_during_training \
        --per_gpu_train_batch_size=3 \
        --per_gpu_eval_batch_size=6 \
        --gradient_accumulation_steps=2 \
        --num_train_epochs=30 \
        --logging_steps=100 \
        --save_steps=1000 \
        --overwrite_output_dir \
        --seed=42

2020-11-30 23:51:31.231443: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
local_rank: 0, node_index: 0, gpu_per_node: 1
11/30/2020 23:51:33 - WARNING - __main__ -   Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: True, 16-bits training: False, world size: 1
Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
11/30/2020 23:51:34 - INFO - __main__ -   [50259, 23748, 995, 50257, 50258]
11/30/2020 23:51:39 - INFO - __main__ -   GPT2Config {
  "_name_or_path": "microsoft/CodeGPT-small-java-adaptedGPT2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50259,
  "embd_pdrop": 0.1,
  "eos_token_id": 50258,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 

# Evaluation

In [ ]:
!python -u run.py \
        --data_dir=../dataset/concode \
        --langs=java \
        --output_dir=../save/concode \
        --pretrain_dir=../save/concode/checkpoint-5000-16.91 \
        --log_file=text2code_concode_eval.log \
        --model_type=gpt2 \
        --block_size=512 \
        --do_eval \
        --logging_steps=100 \
        --seed=42

2020-11-30 16:59:39.000400: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
local_rank: -1, node_index: -1, gpu_per_node: -1
11/30/2020 16:59:44 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False, world size: 1
11/30/2020 16:59:44 - INFO - __main__ -   [50257, 23748, 995, 50260, 50258]
11/30/2020 16:59:49 - INFO - __main__ -   GPT2Config {
  "_name_or_path": "../save/concode/checkpoint-5000-16.91",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50257,
  "embd_pdrop": 0.1,
  "eos_token_id": 50258,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50260,
  "resid_pdrop": 0.1,
  "summ